In [ ]:
import pickle
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import cupy as cp
from cupyx.scipy import ndimage
import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV, SequentialFeatureSelector
from sklearn.model_selection import cross_val_score

# Configs

In [ ]:
pad = 1000
org_rad = 75 
org_pad = 25
# seq gen configs
n_init_orgs = 10
n_total_orgs = 6
n_search = 30
size = 3000 
# sim anneal configs
niter = 1000
move_len = 25
move_decay = 1
random_perturb = 1/5
perturb_decay = .9975

# Model

In [ ]:
model_sigmas = [200, 700]
feats_ixs = np.array([0,1,1,0]).astype(np.bool8)
combined_df = pd.read_csv("datasets/round_1/combined/rnd0_and_1DF.csv")
feats_df = combined_df[['grad200','density700']]
scaler = MinMaxScaler()
X = scaler.fit_transform(feats_df.values)
y = combined_df.cdx2Dipole.values
model = KernelRidge(kernel="rbf").fit(X, y)

# Functions

In [ ]:
def max_gradient(localIm):
    x, y = org_rad, org_rad

    # get pixel intensities in extremes
    xmax = localIm[x + org_rad - 1, y]
    xmin = localIm[x - org_rad, y]
    ymax = localIm[x, y + org_rad - 1]
    ymin = localIm[x, y - org_rad]

    xdiffnorm  = (xmax - xmin) / org_rad 
    ydiffnorm = (ymax - ymin) / org_rad

    # gradient magnitude and vector
    grad = np.array(np.sqrt(xdiffnorm**2 + ydiffnorm**2))
    gradVec = np.array((xdiffnorm, ydiffnorm))
                   
    return grad, gradVec

def extract_features(image, sigma, centroids):
    # gaussian blur
    im = cp.array(image)
    im_blur = ndimage.gaussian_filter(im, sigma=sigma, mode = 'constant')
    
    # extract features per centroid
    feats = []
    for _, (x, y) in enumerate(centroids):
        
        dfeats = im_blur[x-org_rad:x+org_rad, y-org_rad:y+org_rad]
        density = np.mean(dfeats.get())
        grad, _ = max_gradient(dfeats.get())
        feats.append((density, grad))
      
    return np.array(feats)

"""def random_location(image):
    found_valid = False
    
    while not found_valid:
        x, y = np.random.choice(size,2)
        xl, xr, yu, yd = x-org_rad, x+org_rad, y-org_rad, y+org_rad
        # ensure organoid is at least 2 pads worth away from wall and organoids
        min_rdist = org_rad + 2*org_pad
        if xl > min_rdist and yu > min_rdist and xr < size-min_rdist and yd < size-min_rdist:
            dim = len(image)
            xx, yy = np.mgrid[:min_rdist*2, :min_rdist*2]
            zz = (xx - min_rdist) ** 2 + (yy - min_rdist) ** 2
            circle = zz < min_rdist ** 2
            bool_mat = np.pad(circle, ((x-min_rdist, dim-x-min_rdist),(y-min_rdist, dim-y-min_rdist)))
            
            # check if overlapping with organoid
            if np.sum(image[bool_mat]) == 0: 
                found_valid = True
    return x,y"""

def random_location(image):
    found_valid = False
    
    while not found_valid:
        min_dist = 75 + 4*25
        x, y = np.random.choice(size, 2)
        # ensure organoid is at least 2 pads worth away from wall and organoids
        if x > min_dist and x < size-min_dist and y > min_dist and y < size-min_dist:
            bool_mat = draw_circle(x, y, image)
            
            # check if overlapping with other organoids
            if np.sum(image[bool_mat]) == 0: 
                found_valid = True
            else:
                print("collided")
        else:
            print("wall bump")
    return x,y

def evaluate(im_pattern, centroids):
    new_feats = []
    
    for sigma in model_sigmas:
        feats = extract_features(im_pattern, sigma, centroids)
        new_feats.append(feats[:, 0].reshape(-1,1))
        new_feats.append(feats[:, 1].reshape(-1,1))

    new_feats = np.hstack(new_feats)
    model_feats = new_feats[:, feats_ixs]
    model_feats_scaled = scaler.transform(model_feats)
    preds = model.predict(model_feats_scaled)

    reward = np.min(preds)
    # Alternative: preds.mean() - np.abs(np.max(preds) - np.min(preds))
    # Alternative: preds.mean() - np.sqrt(np.var(preds))
    # Alternative: preds.mean()

    return reward, preds, model_feats_scaled

In [ ]:
def generate_image():
   # print("Starting stochastic pattern generation...")
    # create empty pattern
    mask = np.zeros((size, size))
    centroids = []

    # Initialize mask
    for _ in range(n_init_orgs):
        x, y = random_location(mask)
        centroids.append((x,y))
        mask[x-org_rad:x+org_rad, y-org_rad:y+org_rad] = 255
    
    # Sequentially add organoids stochastically
    for _ in tqdm(range(n_total_orgs-1)):
        sim_organoids = []
        for _ in tqdm(range(n_search), leave = False):
            # copy to evaluate independently
            test_mask = mask.copy()
            test_centroids = centroids.copy()

            # sample test location in mask
            x, y = random_location(mask)
            test_centroids.append((x,y))
            test_mask[x-org_rad:x+org_rad, y-org_rad:y+org_rad] = 255
            objective, _, _ = evaluate(test_mask, test_centroids)
            sim_organoids.append((objective, x, y))
        
        # select best simulated organoid
        best_sim_organoid = sorted(sim_organoids)[-1]
        newx, newy = best_sim_organoid[1], best_sim_organoid[2]
        centroids.append((newx, newy))
        mask[newx-org_rad:newx+org_rad, newy-org_rad:newy+org_rad] = 255

    # add the pad at the end
    gen_pattern = np.pad(mask, pad)
    gen_centroids = np.array(centroids) + pad

    return gen_pattern, gen_centroids

# Circles

In [ ]:
"""def draw_circle(x,y, mask):
    dim = len(mask)
    xx, yy = np.mgrid[:org_rad*2, :org_rad*2]
    zz = (xx - org_rad) ** 2 + (yy - org_rad) ** 2
    circle = zz < org_rad ** 2
    print(x,y)
    
    bool_mat = np.pad(circle, ((x-org_rad, dim-x-org_rad),(y-org_rad, dim-y-org_rad)))
    mask[bool_mat] = 255
    return mask"""


In [ ]:
n_init_orgs = 5

In [ ]:
def draw_circle(x,y, orgIm, radius, fill = False, fillVal = 255):
    dim = len(orgIm)
    xx, yy = np.mgrid[:radius*2, :radius*2]
    zz = (xx - radius) ** 2 + (yy - radius) ** 2
    circle = zz < radius ** 2
    
    bool_mat = np.pad(circle, ((x-radius, dim-x-radius),(y-radius, dim-y-radius)))
    if fill:
        orgIm[bool_mat] = fillVal
        return orgIm
    else:
        return bool_mat

def random_location(image):
    found_valid = False
    
    while not found_valid:
        min_dist = 75 + 2*25
        x, y = np.random.choice(size, 2)
        # ensure organoid is at least 2 pads worth away from wall and organoids
        if x > min_dist and x < size-min_dist and y > min_dist and y < size-min_dist:
            bool_mat = draw_circle(x, y, image, min_dist)
            
            # check if overlapping with other organoids
            if np.sum(image[bool_mat]) == 0: 
                found_valid = True
    
    return x,y

mask = np.zeros((size, size))
centroids = []

# Initialize mask
for _ in range(n_init_orgs):
    x, y = random_location(mask)
    centroids.append((x,y))
    mask = draw_circle(x,y, mask, 75, fill = True)

plt.imshow(mask)
plt.show()

In [ ]:
n_init_orgs

In [ ]:
np.unique(mask)

# Circles Train DF

In [ ]:
combined_df = pd.read_csv('datasets/round_1/combined/rnd0_and_1DF.csv')

In [ ]:
temp_df = combined_df[combined_df.patternID == 0] 
temp_df

In [ ]:
temp_df = combined_df[combined_df.patternID == 0] 
x, y = temp_df.dx.values.astype(int), temp_df.dy.values.astype(int)
centroids = np.array(list(zip(x,y)))

def circle_mask(centroids):
    mask = np.zeros((9000, 9000))

    # Initialize mask
    for x, y in centroids:
        mask = draw_circle(x,y, mask, 75, fill = True)

    return mask

mask = circle_mask(centroids)

plt.figure(figsize=(8,8))
plt.imshow(mask)
plt.show()

In [ ]:
def max_gradient(localIm):
    x, y = org_rad, org_rad

    # get pixel intensities in extremes
    xmax = localIm[x + org_rad - 1, y]
    xmin = localIm[x - org_rad, y]
    ymax = localIm[x, y + org_rad - 1]
    ymin = localIm[x, y - org_rad]

    xdiffnorm  = (xmax - xmin) / org_rad 
    ydiffnorm = (ymax - ymin) / org_rad

    # gradient magnitude and vector
    grad = np.array(np.sqrt(xdiffnorm**2 + ydiffnorm**2))
    gradVec = np.array((xdiffnorm, ydiffnorm))
                   
    return grad, gradVec

def extract_features(sigma, centroids):
    mask = circle_mask(centroids)

    # gaussian blur
    im = cp.array(mask)
    im_blur = ndimage.gaussian_filter(im, sigma=sigma, mode = 'constant')
    
    # extract features per centroid
    feats = []
    gradVecs = []
    for _, (x, y) in enumerate(centroids):
        
        dfeats = im_blur[x-org_rad:x+org_rad, y-org_rad:y+org_rad]
        density = np.mean(dfeats.get())
        grad, grad_vec = max_gradient(dfeats.get())

        feats.append((x, y, grad, density))
        gradVecs.append(grad_vec)
      
    return im_blur.get(), mask, np.array(feats), gradVecs

In [ ]:
sigBlurs, sigMasks, sigFeats, sigVecs = [], [], [], []
for sig in range(100, 2000, 100): 
    blur, mask, feats, vecs = extract_features(sig, centroids)
    sigBlurs.append(blur)
    sigMasks.append(mask)
    sigFeats.append(feats)
    sigVecs.append(vecs)
    

In [ ]:

for sig in range(8):
    feats = sigFeats[sig]

    grad_vecs = sigVecs[sig]
    vecs = np.array(grad_vecs)
    norms = np.linalg.norm(vecs, axis = 1)
    vecNormed = vecs/norms.reshape(-1,1)

    blur = sigBlurs[sig]
    mask = sigMasks[sig]

    maskblur = blur*mask

    plt.figure(figsize=(8,8))
    plt.imshow(maskblur)
    plt.title("Mask x Blur Sigma %d" % (100*(sig+1)))
    plt.show()

    for grad, vec in zip(feats, vecNormed):
        for c in range(2, 81):
            newp = grad[:2] + c*vec
            vx = int(newp[0])
            vy = int(newp[1])
            maskblur[vx-5:vx+5, vy-5:vy+5] = 0


    plt.figure(figsize=(8,8))
    plt.imshow(maskblur)
    plt.title("Mask x Blur w/ Grad Vecs Sigma %d" % (100*(sig+1)))
    plt.show()

In [ ]:
temp_df

In [ ]:
circle_feats = np.hstack([sigFeats[i][:, 2:] for i in range(8)])


In [ ]:
circle_feats.shape

In [ ]:
five_cols = temp_df.iloc[:, :5].columns.tolist()
rest_of_cols = temp_df.iloc[:, 5:].columns.tolist()

In [ ]:
combined_df

In [ ]:
patternID = np.unique(combined_df.patternID.values)
allIDs = []
for id in patternID:
    temp_df = combined_df[combined_df.patternID == id] 
    centroids = temp_df.values[:, :2].astype(int)

    sigBlurs, sigMasks, sigFeats, sigVecs = [], [], [], []
    for sig in range(100, 2000, 100): 
        _, _, feats, _ = extract_features(sig, centroids)
        assert(np.all(feats[:, :2] == centroids))
        sigFeats.append(feats[:, 2:])

    featsStacked = np.hstack(sigFeats)
    dfID = np.hstack([temp_df.values[:, :5], featsStacked])
    allIDs.append(dfID)

In [ ]:
circle_combined_df = np.hstack([np.vstack(allIDs), combined_df.values[:, -1].reshape(-1,1)])

In [ ]:
#combined_df.columns
featcols = [
    'grad100', 'density100',
    'grad200', 'density200',
    'grad300', 'density300',
    'grad400', 'density400',
    'grad500', 'density500',
    'grad600', 'density600',
    'grad700', 'density700',
    'grad800', 'density800',
    'grad900', 'density900',
    'grad1000', 'density1000',
    'grad1100', 'density1100',
    'grad1200', 'density1200',
    'grad1300', 'density1300',
    'grad1400', 'density1400',
    'grad1500', 'density1500',
    'grad1600', 'density1600',
    'grad1700', 'density1700',
    'grad1800', 'density1800',
    'grad1900', 'density1900',
]

In [ ]:
newcols = combined_df.columns.tolist()[:5]+featcols+[combined_df.columns.tolist()[-1]]
len(newcols)

In [ ]:
circle_combined_df = pd.DataFrame(circle_combined_df, columns=newcols)
circle_combined_df.to_csv("circle_combined_df.csv", index = False)

# Circle Modeling

In [ ]:
X, y = circle_combined_df.values[:, 5:-1], circle_combined_df.values[:, -1]

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
feat_names = circle_combined_df.iloc[:, 5:-1].columns.values

In [32]:
# prediciting on single degree variables
estimator = KernelRidge(kernel="rbf")

for n in range(1, 6):
    sfs = SequentialFeatureSelector(estimator, n_features_to_select=n, scoring="neg_root_mean_squared_error")
    sfs.fit(X_scaled, y)
    select_df = X_scaled[:, sfs.get_support()]
    scores = cross_val_score(estimator, select_df, y, cv=5, scoring="neg_root_mean_squared_error")
    print(scores.mean(), feat_names[sfs.get_support()])

KeyboardInterrupt: 

In [ ]:
# prediciting on single degree variables
estimator = KernelRidge(kernel="rbf")
sfs = SequentialFeatureSelector(estimator, n_features_to_select=2, scoring="neg_root_mean_squared_error")
sfs.fit(X_scaled, y)
select_df = X_scaled[:, sfs.get_support()]

In [ ]:
plt.figure(figsize = (16,8))
plt.subplot(1,2,1)
plt.scatter(select_df[:, 0], select_df[:, 1], c = np.log1p(y), s = 15)
plt.xlabel("grad200")
plt.ylabel("density700")
plt.gca().set_facecolor((0,0,0))

estimator = KernelRidge(kernel="rbf").fit(select_df, y)

plt.subplot(1,2,2)
plt.scatter(select_df[:, 0], select_df[:, 1], c = estimator.predict(select_df))
plt.xlabel("grad200")
plt.ylabel("density700")
plt.gca().set_facecolor((0,0,0))
plt.show()